In [2]:
from google.cloud import bigquery
from schema import Schema

## Initializing the Client and Job Config

In [3]:
# Before running this line, make sure that you have defined the environment variable...
# ..."GOOGLE_APPLICATION_CREDENTIALS" which points to the JSON file containing authentication key
client = bigquery.Client()

In [3]:
# Initializing the Job_config
job_config = bigquery.LoadJobConfig()
job_config.skip_leading_rows = 1
job_config.max_bad_records = 10
job_config.source_format = bigquery.SourceFormat.CSV
dataset_ref = client.dataset('patstat')

## Creating Tables

In [6]:
# List of Tables to be loaded
[
    'tls201', # TLS201_APPL
    'tls202', # TLS202_APPLN_TITLE
    'tls203', # TLS203_APPLN_ABSTR
    'tls204', # TLS204_APPLN_PRIOR
    'tls205', # TLS205_TECH_REL
    'tls206', # TLS206_PERSON
    'tls207', # TLS207_PERS_APPLN
    'tls209', # TLS209_APPLN_IPC
    'tls210', # TLS210_APPLN_N_CLS
    'tls211', # TLS211_PAT_PUBLN
    'tls212', # TLS212_CITATION
    'tls214', # TLS214_NPL_PUBLN
    'tls215', # TLS215_CITN_CATEG
    'tls216', # TLS216_APPLN_CONTN
    'tls222', # TLS222_APPLN_JP_CLASS
    'tls223', # TLS223_APPLN_DOCUS
    'tls224', # TLS224_APPLN_CPC
    'tls226', # TLS226_PERSON_ORIG
    'tls227', # TLS227_PERS_PUBLN
    'tls229', # TLS229_APPLN_NACE2
    'tls230', # TLS230_APPLN_TECHN_FIELD
    'tls231', # TLS231_INPADOC_LEGAL_EVENT
    'tls801', # TLS801_COUNTRY
    'tls803', # TLS803_LEGAL_EVENT_CODE
    'tls901', # TLS901_TECHN_FIELD_IPC
    'tls902', # TLS902_IPC_NACE2
    'tls904', # TLS904_NUTS
    'tls906'  # TLS906_PERSON
]

# Google Bucket directory address, which contains all data files
#gs_add = 'gs://patstat_2018g/new/'
gs_add = 'gs://patstat_2018g/'

dataset_id = 'PATSTAT'
dataset_ref = client.dataset(dataset_id=dataset_id)

# Loading the tables in the list
for i, table_id in enumerate(tables_list):
    # Creating the table
    print('Step {} has started! {} is creating...'.format(i, table_id))
    table_ref = dataset_ref.table(table_id)
    if table_id in [t.table_id for t in list(client.list_tables(dataset_ref))]:
        client.delete_table(table_ref)

    client.create_table(bigquery.Table(table_ref, schema=getattr(Schema(),table_id)))

    # Adding files to the table from GCP bucket
    table_ref = dataset_ref.table(table_id)
    job_config.schema = getattr(Schema(),table_id)
    
    load_job = client.load_table_from_uri(
        source_uris=gs_add+table_id+'_*.gz',
        destination=table_ref,
        # job_id=job_id,
        job_id_prefix='lgs-',
        job_config=job_config,
    )
    load_job.result()

Step 0 has started! tls204 is creating...


In [ ]:
load_job.errors